### **If you are using Colab for free, we highly recommend you active the T4 GPU hardware accelerator. Our models are designed to run with at least 16GB of RAM, activating T4 will grant the notebook 16GB of GDDR6 RAM as opposed to the 13GB Colab gives automatically.**
### **To active T4:**
### **1. click on the "Runtime" tab**
### **2. click on "Change runtime type"**
### **3. select T4 GPU under Hardware Accelerator**
### **NOTE: there is a weekly usage limit on using T4 for free**

# **Fast Start Example 7 - Model Function Calls** #
This example shows how function calling can be applied to small language models through SLIMs (Structured Language Instruction Models). They accept three main inputs to create a prompt and generate a structured output.

Inputs:

    -- Text passage - this is the core passage or piece of text that you would like the model to assess
    -- Function - classify, extract, generate - this is handled by default by the model class, so usually does not need to be explicitly declared - but is an option for SLIMs that support more than one function
    -- Params - depends upon the model, used to configure/guide the behavior of the function call - optional for some SLIMs

Outputs:

    -- Structured python output, generally either a dictionary or list

The main objectives are to allow small, locally-running models to perform function calling using simple prompts and standardized outputs.

# Discover SLIM tools from Model Catalog


In [3]:
!pip install llmware

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 24.9 MB/s eta 0:00:00


In [4]:
from llmware.models import ModelCatalog

In [5]:
tools = ModelCatalog().list_llm_tools()
tool_map = ModelCatalog().get_llm_fx_mapping()

In [6]:
print("\nList of SLIM model tools (GGUF) in the ModelCatalog\n")
for i, tool in enumerate(tools):
    model_card = ModelCatalog().lookup_model_card(tool_map[tool])
    print(f"{i} - tool: {tool} - "
        f"model_name: {model_card['model_name']} - "
        f"model_family: {model_card['model_family']}")


List of SLIM model tools (GGUF) in the ModelCatalog

0 - tool: ner - model_name: slim-ner-tool - model_family: GGUFGenerativeModel
1 - tool: sentiment - model_name: slim-sentiment-tool - model_family: GGUFGenerativeModel
2 - tool: topics - model_name: slim-topics-tool - model_family: GGUFGenerativeModel
3 - tool: ratings - model_name: slim-ratings-tool - model_family: GGUFGenerativeModel
4 - tool: emotions - model_name: slim-emotions-tool - model_family: GGUFGenerativeModel
5 - tool: nli - model_name: slim-nli-tool - model_family: GGUFGenerativeModel
6 - tool: intent - model_name: slim-intent-tool - model_family: GGUFGenerativeModel
7 - tool: sql - model_name: slim-sql-tool - model_family: GGUFGenerativeModel
8 - tool: answer - model_name: bling-answer-tool - model_family: GGUFGenerativeModel
9 - tool: category - model_name: slim-category-tool - model_family: GGUFGenerativeModel
10 - tool: tags - model_name: slim-tags-tool - model_family: GGUFGenerativeModel
11 - tool: summary - model

# Function calls using different SLIMs

In [15]:
print("\nExecuting Function Call Inferences with SLIMs\n")


Executing Function Call Inferences with SLIMs



Perform sentiment analysis

In [16]:
passage1 = ("This is one of the best quarters we can remember for the industrial sector "
            "with significant growth across the board in new order volume, as well as price "
            "increases in excess of inflation.  We continue to see very strong demand, especially "
            "in Asia and Europe. Accordingly, we remain bullish on the tier 1 suppliers and would "
            "be accumulating more stock on any dips.")

model = ModelCatalog().load_model("slim-sentiment-tool")
response = model.function_call(passage1)

print("sentiment response: ", response['llm_response'])

sentiment response:  {'sentiment': ['positive']}


Named entity recognition

In [17]:
passage2 = "Michael Johnson was a famous Olympic sprinter from the U.S. in the early 2000s."

model = ModelCatalog().load_model("slim-ner-tool")
response = model.function_call(passage2)

print("ner response: ", response['llm_response'])

ner response:  {'people': ['Michael Johnson'], 'location': ['U.S.'], 'organization': [], 'misc': []}


Extract information based on any parameter

In [18]:
passage3 = ("Adobe shares tumbled as much as 11% in extended trading Thursday after the design software maker "
            "issued strong fiscal first-quarter results but came up slightly short on quarterly revenue guidance. "
            "Here’s how the company did, compared with estimates from analysts polled by LSEG, formerly known as Refinitiv: "
            "Earnings per share: $4.48 adjusted vs. $4.38 expected Revenue: $5.18 billion vs. $5.14 billion expected "
            "Adobe’s revenue grew 11% year over year in the quarter, which ended March 1, according to a statement. "
            "Net income decreased to $620 million, or $1.36 per share, from $1.25 billion, or $2.71 per share, "
            "in the same quarter a year ago. During the quarter, Adobe abandoned its $20 billion acquisition of "
            "design software startup Figma after U.K. regulators found competitive concerns. The company paid "
            "Figma a $1 billion termination fee.")

model = ModelCatalog().load_model("slim-extract-tool")
response = model.function_call(passage3, function="extract", params=["revenue growth"])

print("extract response: ", response['llm_response'])

extract response:  {'revenue_growth': ['11% year over year']}


Generate questions, topic, summary, booolean, and tags responses

In [19]:
model = ModelCatalog().load_model("slim-q-gen-tiny-tool", temperature=0.2, sample=True)
response = model.function_call(passage3, params=['multiple choice'])

print("question generation response: ", response['llm_response'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1194: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/38.8k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

q_gen.gguf:   0%|          | 0.00/668M [00:00<?, ?B/s]

question generation response:  {'question': ['What was Figma’s termination fee - (A) $20 billion, (B) $2 billion, (C) $1 billion, or (D) $620 million?']}


In [20]:
model = ModelCatalog().load_model("slim-topics-tool")
response = model.function_call(passage3)

print("topics response: ", response['llm_response'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1194: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

slim-topics.gguf:   0%|          | 0.00/669M [00:00<?, ?B/s]

topics response:  {'topics': ['earnings']}


In [21]:
model = ModelCatalog().load_model("slim-xsum-tool", temperature=0.0, sample=False)
response = model.function_call(passage3)

print("xsum response: ", response['llm_response'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1194: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

slim-xsum.gguf:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

xsum response:  {'xsum': ['Adobe shares tumbled as much as 11% in extended trading Thursday after the design software maker issued strong fiscal first-quarter results but came up slightly short on quarterly revenue guidance.']}


In [25]:
model = ModelCatalog().load_model("slim-boolean-tool")
response = model.function_call(passage3, params=["Did Adobe revenue increase? (explain)"])

print("boolean response: ", response['llm_response'])

boolean response:  {'answer': ['yes'], 'explanation': ['11% year over year.']}


In [26]:
model = ModelCatalog().load_model("slim-tags-tool", temperature=0.0, sample=False)
response = model.function_call(passage3)

print("tags response: ", response['llm_response'])

tags response:  {'tags': '["Adobe", "LSEG", "Figma", "Adobe shares", "March 1", "11", "1.36", "2.71", "20", "1", "620", "1.25", "2.71"]'}


# Using Logits and working with outputs
    1.  Using Logit Information to indicate confidence levels, especially for classifications.
    2.  Using the structured dictionary generated for programmatic handling in a larger process.

---



In [27]:
print("\nExample: using logits and integrating into process\n")


Example: using logits and integrating into process



In [28]:
text_passage = ("On balance, this was an average result, with earnings in line with expectations and "
                "no big surprises to either the positive or the negative.")

sentiment_model = ModelCatalog().load_model("slim-sentiment-tool", get_logits=True)
response = sentiment_model.function_call(text_passage)
analysis = ModelCatalog().logit_analysis(response,sentiment_model.model_card, sentiment_model.hf_tokenizer_name)

print("sentiment response: ", response['llm_response'])

INFO:llmware.models:update: LocalTokenizer - need to fetch tokenizer - tokenizer_tl.json
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1194: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


tokenizer_tl.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

sentiment response:  {'sentiment': ['positive']}


In [29]:
print("\nAnalyzing response")

for keys, values in analysis.items():
    print(f"{keys} - {values}")

sentiment_value = response["llm_response"]["sentiment"]
confidence_level = analysis["confidence_score"]

if "positive" in sentiment_value:
    print("sentiment is positive .... will take 'positive' analysis path ...", sentiment_value)
else:
    print("sentiment is negative .... will take 'negative' analysis path ...", sentiment_value)

if "positive" in sentiment_value and confidence_level > 0.8:
    print("sentiment is positive with high confidence ... ", sentiment_value, confidence_level)


Analyzing response
ryg_string - sentiment': ['positive']}
choices - [{'choice_1': ['pos', 0.506, 1066], 'choice_2': ['neut', 0.381, 17821], 'choice_3': ['negative', 0.11, 22198]}, {'choice_1': ['itive', 1.0, 3321], 'choice_2': ['it', 0.0, 277], 'choice_3': ["']", 0.0, 2033]}]
marker_tokens - [('positive', 1066, 0.506), ('neutral', 17821, 0.381), ('negative', 22198, 0.11)]
low_confidence_choices - [{'choice_1': ['pos', 0.506, 1066], 'choice_2': ['neut', 0.381, 17821], 'choice_3': ['negative', 0.11, 22198]}]
confidence_score - 0.506
sentiment is positive .... will take 'positive' analysis path ... ['positive']
